<a href="https://colab.research.google.com/github/Satishkumar0651/1.-ad_org/blob/master/Ecom_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://www.dropbox.com/s/f85flvz4u9kyp9s/dataset_assignment.zip

--2023-07-28 08:24:50--  https://www.dropbox.com/s/f85flvz4u9kyp9s/dataset_assignment.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/f85flvz4u9kyp9s/dataset_assignment.zip [following]
--2023-07-28 08:24:51--  https://www.dropbox.com/s/raw/f85flvz4u9kyp9s/dataset_assignment.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6b64ee5a938cee5ff8ff116253.dl.dropboxusercontent.com/cd/0/inline/CAsT_2OPwPNZfv6h8TpG2DTm8HMgKSys_3KlRtQfzQ4mTnivtlILHYUnXZd9GbGYY9Lpya3wpP2j-oSdP-t4G-lwat6k_OBnqR42hbYW4ZFnRdAXCs-IwnaaPHFUHbNn3OATbmtfLxs_wKwmm8XA2T5G/file# [following]
--2023-07-28 08:24:51--  https://uc6b64ee5a938cee5ff8ff116253.dl.dropboxusercontent.com/cd/0/inline/CAsT_2OPwPNZfv6h8TpG2DTm8HMgKSys_3KlRtQfzQ4mTnivtlILHYUnXZd9GbGYY9Lpya3w

In [2]:
!unzip dataset_assignment.zip

Archive:  dataset_assignment.zip
replace dataset_assignment/test_set_sequences? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!ls dataset_assignment/

stats  test_set_sequences  train_set_sequences	val_set_sequences


### **Next Movie Prediction**

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

class RNNCCEModel(tf.keras.Model):
    def __init__(self, num_items, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.rnn = tf.keras.layers.GRU(rnn_units, return_sequences=False)
        self.final_layer = tf.keras.layers.Dense(num_items)

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.rnn(x)
        x = self.final_layer(x)
        return x



# Read the data from the file
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines



def preprocess_data(lines, num_movies):
    user_sequences = []
    targets = []

    for line in lines:
        data = line.strip().split()
        user_id = int(data[0])
        movie_ids = [int(data[i]) for i in range(1, len(data), 2)]

        # Append each user sequence with movie IDs (except the last one) to user_sequences
        user_sequences.append(movie_ids[:-1])

        # Append the last movie ID in each user sequence to targets
        targets.append(movie_ids[-1])

    # Pad the sequences so they all have the same length
    user_sequences = pad_sequences(user_sequences)

    # Convert the lists to numpy arrays for TensorFlow
    user_sequences = np.array(user_sequences)
    targets = np.array(targets)

    # Create one-hot encoded targets for Categorical Cross Entropy
    targets = tf.keras.utils.to_categorical(targets, num_classes=num_movies)

    return user_sequences, targets

In [5]:

# Load the data from the files
train_file_path = 'dataset_assignment/train_set_sequences'
validation_file_path = 'dataset_assignment/val_set_sequences'
test_file_path = 'dataset_assignment/test_set_sequences'

train_lines = read_data(train_file_path)
validation_lines = read_data(validation_file_path)
test_lines = read_data(test_file_path)


In [6]:




# Assuming we have the number of unique movies (num_movies) from the dataset
num_movies = 3706

# Preprocess the data for training, validation, and test sets
train_user_sequences, train_targets = preprocess_data(train_lines, num_movies)
validation_user_sequences, validation_targets = preprocess_data(validation_lines, num_movies)
test_user_sequences, test_targets = preprocess_data(test_lines, num_movies)


# Create the training, validation, and test datasets using tf.data.Dataset
batch_size = 32
# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_user_sequences, train_targets)).batch(batch_size)

# Prepare the validation dataset
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_user_sequences, validation_targets)).batch(batch_size)

# Prepare the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test_user_sequences, test_targets)).batch(batch_size)



embedding_dim = 100
rnn_units = 128
num_epochs =  30
batch_sizes =  64
learning_rate =  0.01



def compute_sps(model, dataset):
    total_users = 0
    total_sps = 0.0

    for user_input, targets in dataset:
        # The targets are one-hot encoded, so find the original movie_id of the first target
        target_movie_id = tf.argmax(targets[:, 0])

        #  model's predictions for the user
        predictions = model(user_input)

        # indices of the top-k predicted items for each user
        # We're only interested in the first recommendation, so we set k=1
        top_k_indices = tf.math.top_k(predictions, k=1).indices

        # Check if the first target movie is in the top-k predictions
        sps = tf.reduce_sum(tf.cast(tf.equal(target_movie_id, tf.cast(top_k_indices, tf.int64)), tf.float32))

        total_sps += sps
        total_users += 1

    # Compute the average SPS over all users in the dataset
    average_sps = total_sps / total_users

    return average_sps.numpy()



# Create the RNNCCE model
model = RNNCCEModel(num_movies, embedding_dim, rnn_units)

# Define the loss function and optimizer
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for movie_input, targets in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(movie_input)
            loss = loss_object(targets, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Evaluate the model on the validation set and compute the SPS metric
    validation_sps = compute_sps(model, validation_dataset)
    print(f"Epoch {epoch + 1}, Validation SPS: {validation_sps:.4f}")

Epoch 1, Validation SPS: 0.0000
Epoch 2, Validation SPS: 0.0000
Epoch 3, Validation SPS: 0.0000
Epoch 4, Validation SPS: 0.0000
Epoch 5, Validation SPS: 0.0000
Epoch 6, Validation SPS: 0.0000
Epoch 7, Validation SPS: 0.0000
Epoch 8, Validation SPS: 0.0000
Epoch 9, Validation SPS: 0.0000
Epoch 10, Validation SPS: 0.0000
Epoch 11, Validation SPS: 0.0000
Epoch 12, Validation SPS: 0.0000
Epoch 13, Validation SPS: 0.0000
Epoch 14, Validation SPS: 0.0000
Epoch 15, Validation SPS: 0.0000
Epoch 16, Validation SPS: 0.0000
Epoch 17, Validation SPS: 0.0000
Epoch 18, Validation SPS: 0.0000
Epoch 19, Validation SPS: 0.0000
Epoch 20, Validation SPS: 0.0000
Epoch 21, Validation SPS: 0.0000
Epoch 22, Validation SPS: 0.0000
Epoch 23, Validation SPS: 0.0000
Epoch 24, Validation SPS: 0.0000
Epoch 25, Validation SPS: 0.0000
Epoch 26, Validation SPS: 0.0000
Epoch 27, Validation SPS: 0.0000
Epoch 28, Validation SPS: 0.0000
Epoch 29, Validation SPS: 0.0000
Epoch 30, Validation SPS: 0.0000


### **Next Three Movies Prediction**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

class RNNCCEModel(tf.keras.Model):
    def __init__(self, num_items, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.rnn = tf.keras.layers.GRU(rnn_units, return_sequences=True)  # Now it returns the output for each timestep
        self.final_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_items))  # Apply Dense layer to each timestep output

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.rnn(x)
        # Select only the last three timesteps output for making predictions
        x = x[:, -3:, :]
        x = self.final_layer(x)
        # Apply softmax over the final axis
        x = tf.nn.softmax(x, axis=-1)
        return x



# Read the data from the .txt file
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

In [ ]:
def preprocess_data(lines, num_movies):
    user_sequences = []
    targets = []

    for line in lines:
        data = line.strip().split()
        user_id = int(data[0])
        movie_ids = [int(data[i]) for i in range(1, len(data), 2)]

        # Append each user sequence with movie IDs (except the last three) to user_sequences
        user_sequences.append(movie_ids[:-3])

        # Append the last three movie IDs in each user sequence to targets
        targets.append(movie_ids[-3:])

    # Pad the sequences so they all have the same length
    user_sequences = pad_sequences(user_sequences)
    targets = pad_sequences(targets)

    # Convert the lists to numpy arrays for TensorFlow
    user_sequences = np.array(user_sequences)
    targets = np.array(targets)

    # Create one-hot encoded targets for Categorical Cross Entropy
    targets = tf.keras.utils.to_categorical(targets, num_classes=num_movies)

    return user_sequences, targets

# Load the data from the files
train_file_path = 'dataset_assignment/train_set_sequences'
validation_file_path = 'dataset_assignment/val_set_sequences'
test_file_path = 'dataset_assignment/test_set_sequences'

train_lines = read_data(train_file_path)
validation_lines = read_data(validation_file_path)
test_lines = read_data(test_file_path)

# Assuming we have the number of unique movies (num_movies) from the dataset
num_movies = 3706

# Preprocess the data for training, validation, and test sets
train_user_sequences, train_targets = preprocess_data(train_lines, num_movies)
validation_user_sequences, validation_targets = preprocess_data(validation_lines, num_movies)
test_user_sequences, test_targets = preprocess_data(test_lines, num_movies)


# Create the training, validation, and test datasets using tf.data.Dataset
batch_size = 32
# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_user_sequences, train_targets)).batch(batch_size)

# Prepare the validation dataset
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_user_sequences, validation_targets)).batch(batch_size)

# Prepare the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test_user_sequences, test_targets)).batch(batch_size)


# Define the hyperparameter grid for grid search
embedding_dim = 100
rnn_units = 128
num_epochs =  30
batch_sizes =  64
learning_rate =  0.01

def compute_sps(model, dataset):
    total_users = 0
    total_sps = 0.0

    for user_input, targets in dataset:
        # The targets are one-hot encoded, so find the original movie_id of the last three targets
        target_movie_ids = tf.argmax(targets, axis=-1)  # shape: (batch_size, 3)

        # model's predictions for the user
        predictions = model(user_input)

        # indices of the top-3 predicted items for each user at each timestep
        top_k_indices = tf.math.top_k(predictions, k=3).indices  # shape: (batch_size, 3, 3)

        # Check if the target movie is in the top-k predictions at the corresponding timestep
        sps = tf.reduce_any(tf.equal(tf.expand_dims(target_movie_ids, axis=-1), tf.cast(top_k_indices, tf.int64)), axis=-1)  # shape: (batch_size, 3)
        sps = tf.cast(sps, tf.float32)

        total_sps += tf.reduce_sum(sps)  # Sum over the batch dimension
        total_users += user_input.shape[0]  # Add the batch size to the total number of users

    # Compute the average SPS over all users in the dataset
    average_sps = total_sps / total_users

    return average_sps.numpy()



# Create the RNNCCE model
model = RNNCCEModel(num_movies, embedding_dim, rnn_units)

# Define the loss function and optimizer
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for movie_input, targets in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(movie_input)
            loss = loss_object(targets, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Evaluate the model on the validation set and compute the SPS metric
    validation_sps = compute_sps(model, validation_dataset)
    print(f"Epoch {epoch + 1}, Validation SPS: {validation_sps:.4f}")


Epoch 1, Validation SPS: 0.0500
Epoch 2, Validation SPS: 0.0500
Epoch 3, Validation SPS: 0.0500
Epoch 4, Validation SPS: 0.0500
Epoch 5, Validation SPS: 0.0500
Epoch 6, Validation SPS: 0.0500
Epoch 7, Validation SPS: 0.0500
Epoch 8, Validation SPS: 0.0500
Epoch 9, Validation SPS: 0.0500
Epoch 10, Validation SPS: 0.0500
Epoch 11, Validation SPS: 0.0500
Epoch 12, Validation SPS: 0.0500
Epoch 13, Validation SPS: 0.0500
Epoch 14, Validation SPS: 0.0500
Epoch 15, Validation SPS: 0.0500
Epoch 16, Validation SPS: 0.0500
Epoch 17, Validation SPS: 0.0500
Epoch 18, Validation SPS: 0.0500
Epoch 19, Validation SPS: 0.0500
Epoch 20, Validation SPS: 0.0500
Epoch 21, Validation SPS: 0.0500
Epoch 22, Validation SPS: 0.0500
Epoch 23, Validation SPS: 0.0500
Epoch 24, Validation SPS: 0.0500
Epoch 25, Validation SPS: 0.0500
Epoch 26, Validation SPS: 0.0500
Epoch 27, Validation SPS: 0.0500
Epoch 28, Validation SPS: 0.0500
Epoch 29, Validation SPS: 0.0500
Epoch 30, Validation SPS: 0.0500


In [ ]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.9 MB/s eta 0:00:00


### **Hyperparameter Tuning**

In [ ]:
import kerastuner as kt
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense, TimeDistributed
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

class RNNCCEModel(tf.keras.Model):
    def __init__(self, num_items, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.rnn = tf.keras.layers.GRU(rnn_units, return_sequences=True)
        self.final_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_items))

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.rnn(x)
        x = x[:, -3:, :]
        x = self.final_layer(x)
        x = tf.nn.softmax(x, axis=-1)
        return x

def model_builder(hp):
    num_items = 3706
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    hp_embedding_dim = hp.Int('embedding_dim', min_value = 32, max_value = 512, step = 32)
    model = RNNCCEModel(num_items, hp_embedding_dim, hp_units)
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate)

    model.compile(optimizer = optimizer, loss = loss_object)

    return model


<ipython-input-10-1a6bb2b7b096>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'loss',
                     max_epochs = 10,
                     factor = 3)

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)
tuner.search(train_dataset, epochs = 50, validation_data = validation_dataset, callbacks = [stop_early])

Trial 26 Complete [00h 04m 04s]
loss: 6.835476398468018

Best loss So Far: 4.947273254394531
Total elapsed time: 00h 49m 10s


In [ ]:
tuner

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
print(f"The optimal number of units is {best_hps.get('units')}, the optimal embedding dimension is {best_hps.get('embedding_dim')}, and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.")

The optimal number of units is 256, the optimal embedding dimension is 416, and the optimal learning rate for the optimizer is 0.001.


In [ ]:

embedding_dim = 416
rnn_units = 256
num_epochs =  30
batch_sizes =  64
learning_rate =  0.001
num_movies=3706
# Create the RNNCCE model with optimal hyperparameters
model = RNNCCEModel(num_items=3706, embedding_dim=416, rnn_units=256)

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    for movie_input, targets in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(movie_input)
            loss = loss_object(targets, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Evaluate the model on the validation set and compute the SPS metric
    validation_sps = compute_sps(model, validation_dataset)
    print(f"Epoch {epoch + 1}, Validation SPS: {validation_sps:.4f}")

Epoch 1, Validation SPS: 0.0680
Epoch 2, Validation SPS: 0.0480
Epoch 3, Validation SPS: 0.0480
Epoch 4, Validation SPS: 0.0480
Epoch 5, Validation SPS: 0.0480
Epoch 6, Validation SPS: 0.0480
Epoch 7, Validation SPS: 0.0480
Epoch 8, Validation SPS: 0.0480
Epoch 9, Validation SPS: 0.0480
Epoch 10, Validation SPS: 0.0640
Epoch 11, Validation SPS: 0.0640
Epoch 12, Validation SPS: 0.0640
Epoch 13, Validation SPS: 0.0640
Epoch 14, Validation SPS: 0.0640
Epoch 15, Validation SPS: 0.0640
Epoch 16, Validation SPS: 0.0640
Epoch 17, Validation SPS: 0.0640
Epoch 18, Validation SPS: 0.0640
Epoch 19, Validation SPS: 0.0640
Epoch 20, Validation SPS: 0.0640
Epoch 21, Validation SPS: 0.0640
Epoch 22, Validation SPS: 0.0640
Epoch 23, Validation SPS: 0.0640
Epoch 24, Validation SPS: 0.0640
Epoch 25, Validation SPS: 0.0640
Epoch 26, Validation SPS: 0.0640
Epoch 27, Validation SPS: 0.0640
Epoch 28, Validation SPS: 0.0640
Epoch 29, Validation SPS: 0.0640
Epoch 30, Validation SPS: 0.0640


**SPS on test_dataset**

In [ ]:
# Test the model on the test dataset
test_sps = compute_sps(model, test_dataset)
print(f"Test SPS: {test_sps:.4f}")

Test SPS: 0.0380


In [ ]:
# from google.colab import drive

# # This will prompt for authorization.
# drive.mount('/content/drive')

Mounted at /content/drive


### **Saved Model**

In [ ]:
# model's weights for later use
model.save_weights('./dataset_assignment/RNNCCEModel_weights.h5')

In [ ]:
num_movies = 3706
embedding_dim = 416
rnn_units = 256
model = RNNCCEModel(num_movies, embedding_dim, rnn_units)
import numpy as np

# Create a random tensor
x = np.random.random((1, 10))

# Call the model on the data
model(x)
model.load_weights('RNNCCEModel_weights.h5')

In [ ]:
# Calculate SPS on test dataset
test_sps = compute_sps(model, test_dataset)
print(f"SPS on test dataset: {test_sps:.4f}")

SPS on test dataset: 0.0380


# QnA Section:
Please answer the following questions based on your understanding of your paper.

### Question 1:

You have a set of 3 sequences:
```
seq1 = ['item1', 'item5', 'item3', 'item2', 'item4']
seq2 = ['item10', 'item5', 'item4', 'item8', 'item2', 'item1', 'item3']
seq3 = ['item8', 'item2', 'item5', 'item3', 'item2', 'item10']
```

We take the first 3 items from all these sequences, and feed them in a RNN model.
The model generates outputs for 3 timesteps for all the 3 input sequences.
Following are the model outputs, where out1 corresponds to the output for input sequence seq1, and so on-
```
out1 = ['item2', 'item4', 'item1']   
out2 = ['item2', 'item10', 'item1']   
out3 = ['item2', 'item3', 'item5']   
```

What will be the sps@3 of this RNN model, for the given set of input sequences?

For seq1, the actual following 3 items are ['item5', 'item3', 'item2'], but the model predicted ['item2', 'item4', 'item1']. None of the predicted items match the actual items. Hence, sps for seq1 is 0.

For seq2, the actual following 3 items are ['item4', 'item8', 'item2'], but the model predicted ['item2', 'item10', 'item1']. Here, the first predicted item matches with the third actual item. Hence, sps for seq2 is 1/3.

For seq3, the actual following 3 items are ['item5', 'item3', 'item2'], but the model predicted ['item2', 'item3', 'item5']. Here, the first and second predicted items match the third and second actual items, respectively. However, sps considers only whether the next item is in the recommendation list, not the order of items. Hence, sps for seq3 is 1.

The average sps for these 3 sequences would then be (0 + 1/3 + 1)/3 = 0.44 (approximately).

### Question 2:

Which do you think is the most significant metric for a recommendation system: sps or recall?


Both sps and recall are significant metrics for a recommendation system, and the choice depends on the context. SPS is a metric that emphasizes short-term, next-item prediction, which is crucial for applications like predicting the next click in a web browsing session. On the other hand, recall is a more general measure of how well the recommendation system can predict all the relevant items for a user, without focusing on the sequence in which they occur. Therefore, if sequence matters (e.g., in a playlist or browsing session), sps may be more significant, while in other contexts (like book or movie recommendation), recall might be more relevant.

### Question 3:

How does the paper propose to measure the short term / long term profiling of a recommender system?

The paper proposes measuring the short-term and long-term profiling of a recommender system through a series of experiments involving distinct metrics. The short-term profiling can be assessed using metrics like sps (Short-term Prediction Success), which is a measure of the system's ability to predict the next item in a user's sequence.

Long-term profiling, on the other hand, can be evaluated using metrics that measure the overall accuracy of recommendations over a more extended period, such as recall or precision. These metrics evaluate how well the system predicts all the relevant items for a user, regardless of their sequence.

### Question 4:
Suppose you have a user U who has watched 2*n movies, and you have trained RNN based recommender with first n movies. Now, you have a number N <= n, denoting the number of next items in the user sequence taken to do short-term/long-term profiling.
As per the paper, what impact does increasing N have on this profiling?

As per the paper, increasing the value of N, which represents the number of next items in the user sequence used for short-term/long-term profiling, has several impacts:

Short-term Profiling: When increasing N, the model gets a larger context of user's recent interactions, which could lead to more accurate next-item predictions. However, it's not always the case that a larger N would result in better short-term prediction success (SPS), as the model might start focusing on long-term preferences, thus missing out on capturing the immediate preferences. The optimal value of N for short-term profiling may depend on the specific characteristics of the data and the user's behavior.

Long-term Profiling: Increasing N tends to favor long-term profiling as the model gets more information about the user's overall preferences rather than just their immediate or most recent ones. This could result in a better recall or precision at K (where K is the number of recommendations made), as the model can surface items that align with the user's broader taste.

Remember, these impacts may vary depending on the specifics of the dataset, the users, the items, and the model used. In some cases, too large a context may introduce noise and reduce performance. Hence, careful experimentation and cross-validation are necessary to find the optimal value of N.

'/content'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# ls

dataset_assignment/     __MACOSX/               sample_data/
dataset_assignment.zip  RNNCCEModel_weights.h5  untitled_project/


In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

class RNNCCEModel(tf.keras.Model):
    def __init__(self, num_items, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(num_items, embedding_dim)
        self.rnn = tf.keras.layers.GRU(rnn_units, return_sequences=True)  # Now it returns the output for each timestep
        self.final_layer = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(num_items))  # Apply Dense layer to each timestep output

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.rnn(x)
        # Select only the last ten timesteps output for making predictions
        x = x[:, -10:, :]
        x = self.final_layer(x)
        # Apply softmax over the final axis
        x = tf.nn.softmax(x, axis=-1)
        return x


# Read the data from the .txt file
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

# Read the data from the file
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return lines

def preprocess_data(lines, num_movies):
    sequence_length = 10  # Hard-coded sequence length
    user_sequences = []
    targets = []

    for line in lines:
        data = line.strip().split()
        user_id = int(data[0])
        movie_ids = [int(data[i]) for i in range(1, len(data), 2)]

        # Split each user sequence into input sequences and target sequences
        for i in range(len(movie_ids) - sequence_length - 10):  # Changed the range here
            input_seq = movie_ids[i:i+sequence_length]
            target_seq = movie_ids[i+sequence_length:i+sequence_length+10]  # Get the next 10 movie IDs as targets

            user_sequences.append(input_seq)
            targets.append(target_seq)

    # Pad the sequences so they all have the same length
    user_sequences = pad_sequences(user_sequences)

    # Convert the lists to numpy arrays for TensorFlow
    user_sequences = np.array(user_sequences)

    # Convert target sequences to one-hot encoded arrays
    targets = np.array([tf.keras.utils.to_categorical(target_seq, num_classes=num_movies) for target_seq in targets])

    return user_sequences, targets


In [ ]:

# Load the data from the files
train_file_path = 'dataset_assignment/train_set_sequences'
validation_file_path = 'dataset_assignment/val_set_sequences'
test_file_path = 'dataset_assignment/test_set_sequences'

train_lines = read_data(train_file_path)
validation_lines = read_data(validation_file_path)
test_lines = read_data(test_file_path)

# Assuming we have the number of unique movies (num_movies) from the dataset
num_movies = 3706

# Preprocess the data for training, validation, and test sets
train_user_sequences, train_targets = preprocess_data(train_lines, num_movies)
validation_user_sequences, validation_targets = preprocess_data(validation_lines, num_movies)
test_user_sequences, test_targets = preprocess_data(test_lines, num_movies)

In [ ]:

# Create the training, validation, and test datasets using tf.data.Dataset
batch_size = 32
# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_user_sequences, train_targets)).batch(batch_size)

# Prepare the validation dataset
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_user_sequences, validation_targets)).batch(batch_size)

# Prepare the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test_user_sequences, test_targets)).batch(batch_size)


# Define the hyperparameter grid for grid search
embedding_dim = 100
rnn_units = 128
num_epochs =  30
batch_sizes =  64
learning_rate =  0.01

def compute_sps(model, dataset):
    total_users = 0
    total_sps = 0.0

    for user_input, targets in dataset:
        # The targets are one-hot encoded, so find the original movie_id of the last three targets
        target_movie_ids = tf.argmax(targets, axis=-1)  # shape: (batch_size, 3)

        # model's predictions for the user
        predictions = model(user_input)

        # indices of the top-3 predicted items for each user at each timestep
        top_k_indices = tf.math.top_k(predictions, k=10).indices  # shape: (batch_size, 3, 3)

        # Check if the target movie is in the top-k predictions at the corresponding timestep
        sps = tf.reduce_any(tf.equal(tf.expand_dims(target_movie_ids, axis=-1), tf.cast(top_k_indices, tf.int64)), axis=-1)  # shape: (batch_size, 3)
        sps = tf.cast(sps, tf.float32)

        total_sps += tf.reduce_sum(sps)  # Sum over the batch dimension
        total_users += user_input.shape[0]  # Add the batch size to the total number of users

    # Compute the average SPS over all users in the dataset
    average_sps = total_sps / total_users

    return average_sps.numpy()



# Create the RNNCCE model
model = RNNCCEModel(num_movies, embedding_dim, rnn_units)

# Define the loss function and optimizer
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for movie_input, targets in train_dataset:
        with tf.GradientTape() as tape:
            predictions = model(movie_input)
            loss = loss_object(targets, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Evaluate the model on the validation set and compute the SPS metric
    validation_sps = compute_sps(model, validation_dataset)
    print(f"Epoch {epoch + 1}, Validation SPS: {validation_sps:.4f}")

